In [38]:
# Parameters for offset calculator: 
    # composition: the composition of the steel alloy in dictionary form (see below), in either mass fraction or mole fraction
        # Example: composition = {'C':0.0042, 'Si':0.0026, 'Mn':0.0072, 'Ni':0.0178, 'Mo':0.0031, 'Cr':0.0085, 'V':0.0001, 'Nb':0.0001, 'Cu':0.0008, 'P':0.00006, 'Al':0.00021, 'Ti':0.0001, 'N':0.00008, 'B':0.000005}
        # Elements required for calculations: C, Mn, Si, Ni, Mo, Cr, V
        # But, ensure all alloying elements are included in the dictionary
    # composition_unit: select what the units are for the composition dictionary inputted above
        # Example: composition_unit = 'Mass Fraction', or composition_unit = 'Mole Fraction'
        # Ensure composition inputted is in either mass fraction or mole fraction
    # X: phase fraction of transformation at strain desired
        # Generally X = 0.01 (i.e., 1%) is used for calculating the offset

def offset_calculator(composition, composition_unit, X):
    
    # A dictionary of the molecular weights of elements #
    molec_weight = {'Fe':55.85,'C':12.01,'Si':28.09,'Mn':54.94,'Ni':58.69,'Cr':52,'Mo':95.94,'W':183.85,'Co':58.93,'V':50.94,'Nb':92.91,'Cu':63.55,'Al':26.98,'Ti':47.88,'O':16,'N':14.01,'B':10.81,'P':30.97,'S':32.06}

    # Check if the fractions of the required elements have been inputted, and if not set element composition to 0 #
    for element in ['C', 'Mn', 'Si', 'Ni', 'Mo', 'Cr', 'V']:
        if element in composition.keys():
            pass
        else:
            composition[element] = 0

    # Check if Fe has been inputted, and if not add as 1 - sum(elements) #
    if 'Fe' in composition.keys():
        pass
    else:
        composition['Fe'] = 1 - sum(composition.values())
    
    # Create a composition dictionary in mole fraction, if mass fraction inputted
    if composition_unit == 'Mass Fraction':
        mass_frac = composition.copy()
        moles = {}
        for element in mass_frac:
            moles[element] = mass_frac[element]/molec_weight[element]
        total_moles = sum(moles.values())
        mole_frac = {}
        for element in moles:
            mole_frac[element] = moles[element]/total_moles
    
    # Create a composition dictionary in mass fraction, if mole fraction inputted
    elif composition_unit == 'Mole Fraction':
        mole_frac = composition.copy()
        mass = {}
        for element in mole_frac:
            mass[element] = mole_frac[element]*molec_weight[element]
        total_mass = sum(mass.values())
        mass_frac = {}
        for element in mass:
            mass_frac[element] = mass[element]/total_mass
    
    # if 'composition_unit' not selected 
    else:
        return print('Incorrect input for -composition_unit-')

    # Calculate lattice parameter of martensite #
    a_Fe = 0.28664
    a_mart = a_Fe + ((((a_Fe - (0.0279*mole_frac['C']))**2)*(a_Fe + (0.2496*mole_frac['C'])) -a_Fe**3)/(3*(a_Fe**2))) - (0.003*mole_frac['Si']) + (0.006*mole_frac['Mn']) + (0.007*mole_frac['Ni']) + (0.031*mole_frac['Mo']) + (0.005*mole_frac['Cr']) + (0.0096*mole_frac['V'])

    # Calculate lattice parameter of austenite #
    a_aust = 0.3573 + (3.3*10**(-1)*mass_frac['C']) + (9.5*10**(-3)*mass_frac['Mn']) - (2*10**(-3)*mass_frac['Ni']) + (6*10**(-3)*mass_frac['Cr']) + (3.1*10**(-2)*mass_frac['Mo']) + (1.8*10**(-2)*mass_frac['V'])

    # Calculate offset at desired volume fraction, X #
    e = (((a_aust**(-3))*((2*X*(a_mart**3)) + ((1-X)*(a_aust**3))))**(1/3)) - 1
    return e